<a href="https://colab.research.google.com/github/Alerza/nlp_task/blob/main/fasttext_for_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output 

In [2]:
!pip install mwparserfromhell==0.6.4 multiprocess==0.70.13
clear_output()

In [3]:
!pip install fasttext
!pip install sentencepiece
!pip install datasets
clear_output()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import numpy as np


In [ ]:
from datasets import load_dataset

dataset_simple = load_dataset("olm/wikipedia", language="simple", date="20230320")

In [ ]:
dataset_simple

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 228181
    })
})

In [ ]:
dataset_simple.save_to_disk('/content/drive/MyDrive/DataSet/simpleenWikiDataset.hfd')

In [5]:
from datasets import load_from_disk
dataset_simple = load_from_disk('/content/drive/MyDrive/DataSet/simpleenWikiDataset.hfd')

In [12]:
newdata = dataset_simple['train']['text'][::50]

In [13]:
len(newdata)

4564

In [14]:
with open("data.txt","w") as wr:
    for item in newdata:
        wr.write("\t".join(item.split(" ")) + "\n")

In [ ]:
from datasets import load_dataset
dataset_fa = load_dataset("wiki_summary")
newdata = dataset_fa['train']['article']#[::50]
with open("datafa.txt","w") as wr:
    for item in newdata:
        wr.write("\t".join(item.split(" ")) + "\n")

In [ ]:
# Python program to merge two files 
data = data2 = "" 
  
# Reading data from first file 
with open('data.txt') as fp: 
    data = fp.read() 
with open('datafa.txt') as fp: 
    data2 = fp.read() 
# Merging two files into one another file 
data += "\n"
data += data2 
with open ('/content/drive/MyDrive/DataSet/fasttext/finaltext.txt', 'w') as fp: 
  fp.write(data)

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

In [ ]:
with open('data.txt') as fp: 
    data = fp.read()
    datat = tokenizer.tokenize(data)
with open ('finaltext.txt', 'w') as fp:
  st = ' '.join(str(x) for x in datat) 
  fp.write(st)


In [ ]:
import gc

del newdata
#del data2
#del dataset_simple


gc.collect()

In [5]:
import fasttext
model = fasttext.train_unsupervised('ttf.txt', model='skipgram',dim=2048)

In [3]:
model.save_model('/content/drive/MyDrive/DataSet/myOwnFasttext/fasttext_token_simple_dim2048_cbow.model')

In [6]:
model.save_model('fasttext_simple_dim2048_cbow.model')

In [11]:
import fasttext
model =  fasttext.load_model('/content/fasttext_simple_dim2048_cbow.model')

In [3]:
len(model.words)

6777

In [4]:
import pandas as pd
words = model.get_words()
words_df = pd.DataFrame(words, columns=['token'])
words_df['ftindex'] = words_df.index
words_df

,token,ftindex
0,.,0
1,",",1
2,1,2
3,the,3
4,0,4
...,...,...
6772,오,6772
6773,xture,6773
6774,imen,6774
6775,Қ,6775


In [5]:
import pandas as pd
llama_vocab = pd.read_csv('/content/drive/MyDrive/DataSet/vocab7b.csv')
llama_vocab.rename(columns = {'Unnamed: 0':'llamaids'}, inplace = True)

In [6]:
def remve_under(token):

    cleantokne = str(token).replace('_','')

    return cleantokne

#sample['tokenizer'] = sample['token'].apply(tokne2tokenize)
llama_vocab['token'] = llama_vocab['token'].apply(remve_under)
llama_vocab

,llamaids,token
0,0,<unk>
1,1,<s>
2,2,</s>
3,3,<0x00>
4,4,<0x01>
...,...,...
31995,31995,黃
31996,31996,왕
31997,31997,收
31998,31998,弘


In [ ]:
llama_vocab

In [7]:
vocab_my_merge = pd.merge(llama_vocab,words_df, how='inner', on='token')
vocab_my_merge = vocab_my_merge[['llamaids','ftindex','token']]
vocab_my_merge

,llamaids,ftindex,token
0,131,3292,<0x80>
1,132,3945,<0x81>
2,133,4957,<0x82>
3,134,5499,<0x83>
4,135,3787,<0x84>
...,...,...,...
6806,31764,6311,ɫ
6807,31774,2993,강
6808,31797,6402,ữ
6809,31940,6730,내


In [8]:
vocab_my_merge.to_excel('mymergewithllama.xlsx')

In [9]:
import torch
llamaemmbding = torch.load('/content/drive/MyDrive/DataSet/embeding_tensor_7b_llama.pt')

In [10]:
import numpy as np

def tokne2fasttext(token):
    ids = model.get_word_id(token)
    if ids>= 0 :
        fasttextID = ids
        fasttextEMBD= model.get_input_vector(ids)
        done = 'direct'
    else:
        ctoken = token.replace('▁','')
        ids = model.get_word_id(ctoken)
        if ids>= 0 :
            fasttextID = ids
            fasttextEMBD= model.get_input_vector(ids)
            done = 'rm_'
        else:
            fasttextID = ids
            fasttextEMBD= model[token]
            done = 'NotAvlbe'
    return fasttextID , fasttextEMBD , done

def tokne2llamaEMBD(llamaids):
    llamaEMBD= llamaemmbding[llamaids].detach().numpy()
    return   llamaEMBD



vocab_my_merge[ 'llamaEMBD'] = vocab_my_merge['llamaids'].apply(tokne2llamaEMBD)

#vocab_en_merge[ 'fasttextEMBD'] = vocab_en_merge['ftindex'].apply(tokne2fasttextEMBD)
vocab_my_merge[['fasttextIDs','fasttextEMBD', 'ftdone']] = vocab_my_merge['token'].apply(tokne2fasttext).apply(pd.Series)
vocab_my_merge

,llamaids,ftindex,token,llamaEMBD,fasttextIDs,fasttextEMBD,ftdone
0,131,3292,<0x80>,"[0.03186, -0.006954, 0.0002095, 0.01637, -0.03...",3292,"[0.011649349, 0.015273062, -0.0074721617, -0.0...",direct
1,132,3945,<0x81>,"[-0.04773, -0.013245, 0.01613, -0.0408, 0.0012...",3945,"[-0.022309244, 0.012107835, 0.007314005, 0.004...",direct
2,133,4957,<0x82>,"[-0.03427, -0.03317, 0.010025, 0.0259, -0.0402...",4957,"[-0.0042778826, 0.0010679914, 0.000117177304, ...",direct
3,134,5499,<0x83>,"[-0.003319, -0.046, 0.05008, 0.001102, 0.02438...",5499,"[0.0033546304, -0.008791739, -0.004608774, 0.0...",direct
4,135,3787,<0x84>,"[-0.01136, -0.01348, 0.02917, -0.02051, -0.041...",3787,"[-0.010090494, -0.02165715, -0.004360972, 0.01...",direct
...,...,...,...,...,...,...,...
6806,31764,6311,ɫ,"[0.015114, -0.01897, 0.001937, -0.002653, -0.0...",6311,"[-0.06185022, -0.03138776, 0.020322416, -0.046...",direct
6807,31774,2993,강,"[0.00871, -0.02776, -0.0045, -0.00292, 0.00566...",2993,"[-0.055514373, -0.037218902, 0.026422562, -0.0...",direct
6808,31797,6402,ữ,"[-0.0228, 0.007797, 0.001909, 0.01508, -0.0085...",6402,"[0.0059084026, 0.0026131659, -0.046447434, -0....",direct
6809,31940,6730,내,"[-0.03458, -0.01165, 0.017, -0.0177, 0.0371, -...",6730,"[-0.030507058, -0.01541425, -0.008100231, -0.0...",direct


In [7]:
vocab_my_merge.iloc[500]['token']

'ext'

In [11]:
i = 2546
print(vocab_my_merge.iloc[i]['token'])
w = model.get_nearest_neighbors(vocab_my_merge.iloc[i]['token'])
ww = [nn for n,nn in w]
print(ww)
print(w)
llamaEMBD_l ,fasttextEMBD_l = [],[]
for ii in ww:
  indx=vocab_my_merge[vocab_my_merge['token']==ii].index
  llamaEM = vocab_my_merge.iloc[indx]['llamaEMBD']
  llamaEMBD_l.append(llamaEM)
  fasttextEM = vocab_my_merge.iloc[indx]['fasttextEMBD']
  fasttextEMBD_l.append(fasttextEM)

print(llamaEMBD_l)
print(fasttextEMBD_l)

property
['properties', 'Properties', 'prop', 'limited', 'limits', 'gravity', 'mediate', 'proof', 'prob', 'drop']
[(0.8819065093994141, 'properties'), (0.7656651735305786, 'Properties'), (0.7232706546783447, 'prop'), (0.691635251045227, 'limited'), (0.687454879283905, 'limits'), (0.6833981871604919, 'gravity'), (0.6765086054801941, 'mediate'), (0.6722491383552551, 'proof'), (0.6699507236480713, 'prob'), (0.6641247868537903, 'drop')]
[3607    [-0.009834, 0.03958, 0.006504, 0.006252, 0.010...
Name: llamaEMBD, dtype: object, 3715    [0.00738, 0.00298, -0.004723, 0.02216, -0.0271...
Name: llamaEMBD, dtype: object, 2768    [-0.01417, -0.01405, -0.05426, -0.02243, 0.019...
Name: llamaEMBD, dtype: object, 6467    [-0.02402, 0.012215, 0.02562, 0.02313, 0.00473...
Name: llamaEMBD, dtype: object, 3852    [-0.01002, 0.01762, -0.00826, 0.03363, -0.0157...
Name: llamaEMBD, dtype: object, 5753    [-0.01309, 0.01857, -0.001501, 0.00528, 0.0051...
Name: llamaEMBD, dtype: object, 4141    [0.0199, -0.00

In [9]:
px = pd.DataFrame()
px['token'] = vocab_my_merge['token']

In [7]:
from sklearn import preprocessing

X = preprocessing.normalize(X)
y = preprocessing.normalize(y.detach().numpy())

In [14]:
word = 'continue'

def calc_similarity(token):
    embdbase = model.get_word_vector(word)
    embd = model.get_word_vector(str(token))
    smltry = (np.dot(embd,embdbase)).item()
    return smltry

px['similarity'] = px['token'].apply(calc_similarity)

px.sort_values(['similarity'], ascending=False).head(20)

,token,similarity
5225,continue,5.613620
5339,continu,5.528639
604,cont,5.378687
3937,temper,4.904650
3331,contr,4.738204
4828,reduce,4.607238
1157,ential,4.459343
4589,basic,4.350661
825,value,4.315592
3917,diff,4.278366


In [99]:
from sklearn import preprocessing
#X = torch.load('/content/tensor_X.pt')
X = preprocessing.normalize(X)


In [25]:
i = 5331
print(vocab_my_merge.iloc[i]['token'])

def calc_similarity(token):
    embdbase = X[i]
    embd = X[token]
    smltry = (np.dot(embd,embdbase)).item()
    return smltry

px['similarity'] = px.index.map(calc_similarity)

px.sort_values(['similarity'], ascending=False).head(20)

algorithm


,token,similarity
5331,algorithm,1.000000
3835,orithms,0.815893
2191,",'",0.787126
5763,methods,0.785984
4077,prefix,0.775756
3226,HE,0.774977
1893,false,0.765350
6114,digit,0.759255
518,""":",0.749483
4589,basic,0.748890


In [26]:
#@title
#y = preprocessing.normalize(y.detach().numpy())
i = 5331
print(vocab_my_merge.iloc[i]['token'])

def calc_similarity(token):
    embdbase = y[i]
    embd = y[token]
    smltry = (np.dot(embd,embdbase)).item()
    return smltry

px['similarity'] = px.index.map(calc_similarity)

px.sort_values(['similarity'], ascending=False).head(20)

algorithm


,token,similarity
5331,algorithm,1.000000
3835,orithms,0.320109
5611,Alg,0.232672
4289,theorem,0.209331
1890,figure,0.202061
3218,alg,0.192669
6472,Machine,0.176764
5310,software,0.168277
1167,equation,0.163490
1069,table,0.158781


In [72]:
np.dot(model.get_word_vector('continu'),model.get_word_vector('continu'))

19.041481

In [78]:
np.dot(y[1959].detach().numpy(),y[5419].detach().numpy())

0.1178751

In [44]:
tnsdataset[5419]

(tensor([-0.0212,  0.0089, -0.0247,  ...,  0.0198, -0.0225,  0.0369]),
 tensor([-0.0158,  0.0063,  0.0161,  ...,  0.0139,  0.0273,  0.0122],
        grad_fn=<SelectBackward0>))

In [37]:
for m in llamaEMBD_l:
  llamaEMBD_l[m].values

array(['[-0.01584   0.006325  0.01605  ...  0.01392   0.0273    0.01215 ]'],
      dtype=object)

In [10]:
vocab_my_merge.to_excel('merge_myfast_llama.xlsx')

In [9]:
import fasttext
model =  fasttext.load_model('/content/fasttext_simple_dim2048_cbow.model')

In [ ]:
import pandas as pd
vocab_my_merge = pd.read_excel('merge_myfast_llama.xlsx')
import fasttext
model =  fasttext.load_model('fasttext_simple_dim2048_cbow.model')


In [1]:
import torch
llamaemmbding = torch.load('/content/drive/MyDrive/DataSet/embeding_tensor_7b_llama.pt')

In [2]:
import pandas as pd
vocab_my_merge = pd.read_excel('/content/mymergewithllama.xlsx')

In [ ]:
for index, row in vocab_my_merge.iterrows():
    print(row['token'])
    model.get_word_vector(str(row['token']))

In [65]:
model.get_word_vector('nan')

array([ 0.08229809, -0.04106521, -0.0765834 , ...,  0.23380236,
       -0.24653788,  0.07557469], dtype=float32)

In [3]:
import torch
from torch.utils.data import TensorDataset

X = torch.empty(0, 2048)
y = torch.empty(0, 4096)

for index, row in vocab_my_merge.iterrows():
    ##row_tensor_X = torch.tensor(model.get_word_vector(str(row['token'])))
    #X = torch.cat((X, row_tensor_X.unsqueeze(0)), dim=0)

    row_tensor_y = llamaemmbding[row['llamaids']]
    y = torch.cat((y, row_tensor_y.unsqueeze(0)), dim=0)

In [22]:
torch.save(X, 'tensor_X.pt')

In [4]:
torch.save(y, 'tensor_y.pt')

In [5]:
X = torch.load('/content/tensor_X.pt')
y = torch.load('/content/tensor_y.pt')
tnsdataset = TensorDataset(X, y)
torch.save(tnsdataset, '/content/drive/MyDrive/DataSet/fasttext/newSkiptnsdataset.pt')

In [70]:
np.dot(X[0],X[0])

74.56377